In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(37) # 使得每次运行得到的随机数都一样

In [2]:
# 准备数据集
import cv2,itertools,pickle,os
from cv2 import xfeatures2d
from glob import glob

class DataSet:
    
    def __init__(self,img_folder,cluster_model_path,img_ext='jpg',max_samples=12,clusters_num=32):
        self.img_folder=img_folder
        self.cluster_model_path=cluster_model_path
        self.img_ext=img_ext
        self.max_samples=max_samples
        self.clusters_num=clusters_num
        self.img_paths=self.__get_img_paths()
        self.all_img_paths=[list(item.values())[0] for item in self.img_paths]
        self.cluster_model=self.__load_cluster_model()
            
    def __get_img_paths(self):
        folders=glob(self.img_folder+'/*-*') # 由于图片文件夹的名称是数字+‘-’开头，故而可以用这个来获取
        img_paths=[]
        for folder in folders:
            class_label=folder.split('\\')[-1]
            img_paths.append({class_label:glob(folder+'/*.'+self.img_ext)}) 
            # 每一个元素都是一个dict，key为文件夹名称，value为该文件夹下所有图片的路径组成的list
        return img_paths
    
    def __get_image(self,img_path,new_size=200):
        def resize_img(image,new_size):
            '''将image的长或宽中的最小值调整到new_size'''
            h,w=image.shape[:2]
            ratio=new_size/min(h,w)
            return cv2.resize(image,(int(w*ratio),int(h*ratio)))
        
        image=cv2.imread(img_path)
        return resize_img(image,new_size)
    
    def __img_sift_features(self,image):
        '''
        提取图片image中的Star特征的关键点，然后用SIFT特征提取器进行计算，
        得到N行128列的矩阵，每幅图中提取的Star特征个数不一样，故而N不一样，
        但是经过SIFT计算之后，特征的维度都变成128维。
        返回该N行128列的矩阵
        '''
        keypoints=xfeatures2d.StarDetector_create().detect(image)
        gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _,feature_vectors=xfeatures2d.SIFT_create().compute(gray,keypoints)
        return feature_vectors
    
    def __calc_imgs_features(self,img_path_list):
        '''获取多张图片的特征矢量，这些特征矢量是合并到一起的，最终组成M行128列的矩阵，返回该矩阵.
        此处的M是每张图片的特征矢量个数之和，即N1+N2+N3....'''
        img_paths=list(itertools.chain(*img_path_list)) # 将多层list展开
        feature_vectors=[]
        [feature_vectors.extend(self.__img_sift_features(self.__get_image(img_path))) for img_path in img_paths]
        return feature_vectors
    
    def __create_save_Cluster(self):
        '''由于folders中含有大量图片，故而取一小部分（max_samples）图片来做K-means聚类。
        '''
        # 获取要进行聚类的小部分图片的路径
        cluster_img_paths=[list(item.values())[0][:self.max_samples] for item in self.img_paths]
        feature_vectors=self.__calc_imgs_features(cluster_img_paths)
        cluster_model = KMeans(self.clusters_num,  # 建立聚类模型
                        n_init=10,
                        max_iter=10, tol=1.0)
        cluster_model.fit(feature_vectors) # 对聚类模型进行训练
        # 将聚类模型保存，以后就不需要再训练了。
        with open(self.cluster_model_path,'wb+') as file:
            pickle.dump(cluster_model,file)
        print('cluster model is saved to {}.'.format(self.cluster_model_path))
        return cluster_model
    
    def __map_feature_to_cluster(self,img_path):
        '''从单张图片中提取Star特征矩阵（N行128列），
        再将该特征矩阵通过K-means聚类算法映射到K个类别中，每一行特征映射到一个簇中，得到N个簇标号的向量，
        统计每一个簇中出现的特征向量的个数，相当于统计词袋中某个单词出现的频次。
        '''
        img_feature_vectors=self.__img_sift_features(self.__get_image(img_path)) # N 行128列
        cluster_labels=self.cluster_model.predict(img_feature_vectors) 
        # 计算这些特征在K个簇中的类别，得到N个数字，每个数字是0-31中的某一个，代表该Star特征属于哪一个簇
        # eg [30 30 30  6 30 30 23 25 23 23 30 30 16 17 31 30 30 30  4 25]
        
        # 统计每个簇中特征的个数
        vector_nums=np.zeros(self.clusters_num) # 32个元素
        for num in cluster_labels:
            vector_nums[num]+=1
        
        # 将特征个数归一化处理：得到百分比而非个数
        sum_=sum(vector_nums)
        return [vector_nums/sum_] if sum_>0 else [vector_nums] # 一行32列，32 个元素组成的list
    
    def __calc_imgs_clusters(self,img_path_list):
        '''获取多张图片的视觉码本，将这些视觉码本组成一个P行32列的矩阵，P是图片张数，32是聚类的类别数。
        返回该P行32列的矩阵'''
        img_paths=list(itertools.chain(*img_path_list)) # 将多层list展开
        code_books=[]
        [code_books.extend(self.__map_feature_to_cluster(img_path)) for img_path in img_paths]
        return code_books
    
    def __load_cluster_model(self):
        '''从cluster_model_path中加载聚类模型，返回该模型，如果不存在或出错，则调用函数准备聚类模型'''
        cluster_model=None
        if os.path.exists(self.cluster_model_path):
            try:
                with open(self.cluster_model_path, 'rb') as f:
                    cluster_model = pickle.load(f)
            except:
                pass
        if cluster_model is None: 
            print('No valid model found, start to prepare model...')
            cluster_model=self.__create_save_Cluster()
        return cluster_model
    
    def get_img_code_book(self,img_path):
        '''获取单张图片的视觉码本，即一行32列的list，每个元素都是对应特征出现的频率'''
        return self.__map_feature_to_cluster(img_path)
    def get_imgs_code_books(self,img_path_list):
        '''获取多张图片的视觉码本，即P行32列的list，每个元素都是对应特征出现的频率'''
        return self.__calc_imgs_clusters(img_path_list)
    def get_all_img_code_books(self):
        '''获取img_folder中所有图片的视觉码本'''
        return self.__calc_imgs_clusters(self.all_img_paths)
    def get_img_labels(self):
        '''获取img_folder中所有图片对应的label，可以从文件夹名称中获取'''
        img_paths=list(itertools.chain(*self.all_img_paths)) 
        return [img_path.rpartition('-')[0].rpartition('\\')[2] for img_path in img_paths]  
    def prepare_dataset(self):
        '''获取img_folder中所有图片的视觉码本和label，构成数据集'''
        features=self.get_all_img_code_books()
        labels=self.get_img_labels()
        return np.c_[features,labels]

In [3]:
dataset=DataSet('E:\PyProjects\DataSet\FireAI\/training_images','./cluster_model1')
# dataset.create_save_Cluster('./k_model1')
# code_book=dataset.get_all_img_code_books()
# print(np.array(code_book).shape)
prepared=dataset.prepare_dataset()
print(prepared.shape)
print(prepared[:2]) # 检查没错，将准备好的数据集保存一下
# img_list=[['E:\\PyProjects\\DataSet\\FireAI\\/training_images\\0-airplanes\\0001.jpg', 
#           'E:\\PyProjects\\DataSet\\FireAI\\/training_images\\0-airplanes\\0002.jpg', 
#           'E:\\PyProjects\\DataSet\\FireAI\\/training_images\\0-airplanes\\0003.jpg']]
# code_book=dataset.get_imgs_code_books(img_list)

df=pd.DataFrame(prepared)
df.to_csv('./prepared_set.txt')

(60, 33)
[['0.0' '0.4' '0.0' '0.05' '0.0' '0.0' '0.0' '0.05' '0.0' '0.0' '0.0'
  '0.1' '0.3' '0.0' '0.0' '0.0' '0.0' '0.05' '0.0' '0.0' '0.0' '0.0'
  '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.0' '0.05' '0']
 ['0.0' '0.02702702702702703' '0.0' '0.0' '0.0' '0.0'
  '0.02702702702702703' '0.02702702702702703' '0.0' '0.05405405405405406'
  '0.08108108108108109' '0.05405405405405406' '0.40540540540540543' '0.0'
  '0.0' '0.0' '0.0' '0.05405405405405406' '0.0' '0.0'
  '0.05405405405405406' '0.0' '0.0' '0.0' '0.02702702702702703' '0.0'
  '0.0' '0.0' '0.08108108108108109' '0.0' '0.10810810810810811' '0.0' '0']]


In [4]:
# 极端随机森林分类器
from sklearn.ensemble import ExtraTreesClassifier

class CLF_Model:
    
    def __init__(self,n_estimators=100,max_depth=16):
        self.model=ExtraTreesClassifier(n_estimators=n_estimators, 
                max_depth=max_depth, random_state=12)
    def fit(self,train_X,train_y):
        self.model.fit(train_X,train_y)
    def predict(self,newSample_X):
        return self.model.predict(newSample_X)
        

In [5]:
dataset_df=pd.read_csv('./prepared_set.txt',index_col=[0])
dataset_X,dataset_y=dataset_df.iloc[:,:-1].values,dataset_df.iloc[:,-1].values
model=CLF_Model()
model.fit(dataset_X,dataset_y)

In [6]:
# 用训练好的model预测新图片，看看它属于哪一类
new_img1='E:\PyProjects\DataSet\FireAI/test0.jpg'
img_code_book=dataset.get_img_code_book(new_img1)
predicted=model.predict(img_code_book)
print(predicted)

new_img2='E:\PyProjects\DataSet\FireAI/test1.jpg'
img_code_book=dataset.get_img_code_book(new_img2)
predicted=model.predict(img_code_book)
print(predicted)

new_img3='E:\PyProjects\DataSet\FireAI/test2.jpg'
img_code_book=dataset.get_img_code_book(new_img3)
predicted=model.predict(img_code_book)
print(predicted)

[0]
[1]
[2]
